<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/ejercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Introduccion/1_TiposDatos/ejercicio/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>


# Ejercicios I

## 1- Cambiar un texto

Necesitamos mostrar en nuestra web la sinopsis de las películas. El problema es que tenemos los textos separados por pipes ("|") en lugar de saltos de línea.

In [1]:
string = 'Sinopsis | Marty McFly, un típico adolescente americano de los años ochenta, '\
'es accidentalmente enviado de vuelta a 1955 en una "máquina del tiempo" realizada con'\
'un DeLorean inventada por un científico un poco loco. | En este viaje, Marty debe '\
'asegurarse de que sus padres se encuentren y se enamoren, para que pueda volver a su tiempo. '

In [2]:
print(string)

Sinopsis | Marty McFly, un típico adolescente americano de los años ochenta, es accidentalmente enviado de vuelta a 1955 en una "máquina del tiempo" realizada conun DeLorean inventada por un científico un poco loco. | En este viaje, Marty debe asegurarse de que sus padres se encuentren y se enamoren, para que pueda volver a su tiempo. 


## 2 - Crear un acrónimo

Ahora nos piden crear un acrónimo (una palabra compuesta por la primera letra de cada palabra) de cada título. Pero antes de eso es necesario que transformemos los títulos a "title case" porque no todos van a llegar prolijos. Entonces, el acrónimo de "Volver al futuro" debería ser "VAF".


In [3]:
titulo = 'Volver al futuro'

In [4]:
# 1. Transformar el string a "title case"
# 1. Transformar el string a "title case"

titulo = titulo.title()

In [5]:
# 2. Crear una lista con todas las palabras del string
# 2. Crear una lista con todas las palabras del string

lista_palabras = titulo.split(' ')

In [6]:
# 3. Recorrer la lista y agregar al acrónimo la primera letra de cada palabra

# 3. Recorrer la lista y agregar al acrónimo la primera letra de cada palabra

acronimo = ''
for palabra in lista_palabras:
    acronimo = acronimo + palabra[0]

print(acronimo)

VAF


## 3 - Evaluar si un número es par o impar

Con que operación podríamos evaluar si un número es par o impar

In [7]:
numero = 3

if numero % 2 == 0: # Recordemos que el operador '%' devuelve el resto de la división.
  print('El número es par.')
else:
  print('El número es impar.')

El número es impar.


## 4 - Dividir pares e impartes  

Dado un rango de valores enteros con distancia de una unidad

```
range(4,10)
```

separar en listas los números pares e impartes iterando por cada elemento del intervalo

In [8]:
range(4,10)

range(4, 10)

In [9]:
pares = []
impares = []

for i in range(4,10):
  if i%2 == 0:
    pares.append(i)
  else:
    impares.append(i)

print('pares:', pares, 'impares:', impares)

pares: [4, 6, 8] impares: [5, 7, 9]


In [10]:
# Otra posibilidad es utilizar listas de comprensión

pares = [i for i in range(4,10) if i%2 == 0]
impares = [i for i in range(4,10) if i%2 != 0]

print('pares:', pares, 'impares:', impares)

pares: [4, 6, 8] impares: [5, 7, 9]
